# Text Classification Notebook

In [ ]:
!pip install --upgrade pip
!pip install tensorflow
!pip install pandas
!pip install numpy
!pip install PySastrawi
!pip install sklearn

In [1]:
# Machine Learning related imports
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input

In [2]:
# Dataset and generic imports
import pandas as pd
import numpy as np
import sklearn
import os
import string

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [3]:
# Load dataset
filepath = os.getcwd()
datasetpath =  os.path.join(filepath, "cleaned_data", "laporanencoded.csv")

github_url = "https://raw.githubusercontent.com/Capstone-Project-B21-CAP0113/ml-tf/main/laporan/cleaned_data/laporanencoded.csv"

laporan = pd.read_csv(datasetpath, encoding="ISO-8859-1")
# Print dataset shape
print(laporan.shape)
# Print dataset head
laporan.head()


(2852, 27)


,text,perselisihan,infrastruktur,pemerintah,kesehatan,teknologi,administrasi,fasilitas,lingkungan,ketertiban,...,air,pendidikan,kebersihan,sosial,wisata,sara,pencurian,korupsi,bbm,keuangan
0,mohon bantuannya untuk menormalkan sistem di l...,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ass pak gubsaya perangkat desa karangsari kec ...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,selamat sore bapak gubernur atau yang mewakili...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,assalamualaikum pak gub saya pengurus paguyuba...,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,lapor saya okada arle sandi email okadaarlegm...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Split text and labels
label_list = [
    "perselisihan",
    "infrastruktur",
    "pemerintah",
    "kesehatan",
    "teknologi",
    "administrasi",
    "fasilitas",
    "lingkungan",
    "ketertiban",
    "listrik",
    "bahaya",
    "lainnya",
    "pungli",
    "ilegal",
    "lalulintas",
    "bencana",
    "air",
    "pendidikan",
    "kebersihan",
    "sosial",
    "wisata",
    "sara",
    "pencurian",
    "korupsi",
    "bbm",
    "keuangan"
] 
x = laporan["text"]
y = laporan[label_list]

for i in label_list:
    print("{}: {}".format(i, (laporan[i] == 1).sum()))

perselisihan: 16
infrastruktur: 353
pemerintah: 68
kesehatan: 80
teknologi: 23
administrasi: 693
fasilitas: 73
lingkungan: 114
ketertiban: 87
listrik: 43
bahaya: 22
lainnya: 139
pungli: 106
ilegal: 47
lalulintas: 42
bencana: 58
air: 85
pendidikan: 134
kebersihan: 9
sosial: 181
wisata: 13
sara: 7
pencurian: 6
korupsi: 39
bbm: 1
keuangan: 82


In [5]:
# Text head
x.head()

0    mohon bantuannya untuk menormalkan sistem di l...
1    ass pak gubsaya perangkat desa karangsari kec ...
2    selamat sore bapak gubernur atau yang mewakili...
3    assalamualaikum pak gub saya pengurus paguyuba...
4    lapor saya okada arle sandi email  okadaarlegm...
Name: text, dtype: object

In [6]:
# Label head
y.head()

,perselisihan,infrastruktur,pemerintah,kesehatan,teknologi,administrasi,fasilitas,lingkungan,ketertiban,listrik,...,air,pendidikan,kebersihan,sosial,wisata,sara,pencurian,korupsi,bbm,keuangan
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()

stopword_factory = StopWordRemoverFactory()
stopword = stopword_factory.create_stop_word_remover()

for i in range(len(x)):
    x[i] = stopword.remove(x[i])
    x[i] = stemmer.stem(x[i])

x.head()

<ipython-input-7-9d2cddb636c2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = stopword.remove(x[i])
<ipython-input-7-9d2cddb636c2>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = stemmer.stem(x[i])


0    bantu normal sistem lpse kabupaten demak mana ...
1    ass gubsaya perangkat desa karangsari kec kara...
2    selamat sore gubernur mewakilisaya ahmad rofiq...
3    assalamualaikum gub urus paguyuban dagang puja...
4    lapor okada arle sandi email okadaarlegmailcom...
Name: text, dtype: object

In [8]:
# Tokenize and pad text
vocab_size = 2000
embedding_dim = 16
max_length = 300
trunc_type = "post"
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(x)

sequences = tokenizer.texts_to_sequences(x)

padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

In [9]:
# Shuffle, batch and separate data into train, dev and test
BUFFER_SIZE = 1000
BATCH_SIZE = 32
DATASET_SIZE = len(x)


tx = tf.convert_to_tensor(padded)
ty = tf.convert_to_tensor(y)

dataset = tf.data.Dataset.from_tensor_slices((tx, ty))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

train_set = dataset.take(int(0.8 * len(dataset)))
test_set = dataset.skip(int(0.8 * len(dataset)))
test_set = dataset.take(int(0.2 * len(dataset))) 

print(len(train_set))
print(len(test_set))

72
18


In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 128),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(26, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         256000    
_________________________________________________________________
bidirectional (Bidirectional (None, None, 256)         263168    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               164352    
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 26)                8

In [11]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
NUM_EPOCHS = 75
history = model.fit(train_set, epochs=NUM_EPOCHS, validation_data=test_set)

Epoch 1/75
72/72 [==============================] - 129s 1s/step - loss: 0.2492 - accuracy: 0.2300 - val_loss: 0.1210 - val_accuracy: 0.2344
Epoch 2/75
72/72 [==============================] - 105s 1s/step - loss: 0.1271 - accuracy: 0.2235 - val_loss: 0.1180 - val_accuracy: 0.2483
Epoch 3/75
72/72 [==============================] - 118s 2s/step - loss: 0.1258 - accuracy: 0.2365 - val_loss: 0.1165 - val_accuracy: 0.2378
Epoch 4/75
72/72 [==============================] - 112s 2s/step - loss: 0.1255 - accuracy: 0.2461 - val_loss: 0.1192 - val_accuracy: 0.2344
Epoch 5/75
72/72 [==============================] - 117s 2s/step - loss: 0.1261 - accuracy: 0.2435 - val_loss: 0.1178 - val_accuracy: 0.2378
Epoch 6/75
72/72 [==============================] - 103s 1s/step - loss: 0.1259 - accuracy: 0.2378 - val_loss: 0.1186 - val_accuracy: 0.2483
Epoch 7/75
72/72 [==============================] - 104s 1s/step - loss: 0.1251 - accuracy: 0.2448 - val_loss: 0.1181 - val_accuracy: 0.2274
Epoch 8/75
72

In [13]:
def treatinput(inp):
    # Make string into lowercase string
    treated = inp.lower()
    # remove punctuation
    treated = treated.translate(str.maketrans("","",string.punctuation))
    # remove trailing whitespace
    treated = treated.strip()
    # Remove stopwords
    treated = stopword.remove(treated)
    # Stem string
    treated = stemmer.stem(treated)
    return treated

In [ ]:
# sample inferrece
sample_laporan = "Listrik di kampung kami sudah mati selama 3 hari"
treated_input = treatinput(sample_laporan)
pad = pad_sequences(tokenizer.texts_to_sequences([treated_input]), maxlen=max_length, truncating=trunc_type)
prediction = model.predict(pad)

res = dict(zip(label_list, prediction[0]))

In [120]:
for i in res.keys():
    # if(res[i] > 0.6):
    print("{} {:.5f}".format(i, res[i]))

perselisihan 0.00000
infrastruktur 0.00155
pemerintah 0.01193
kesehatan 0.00000
teknologi 0.00030
administrasi 0.00000
fasilitas 0.02860
lingkungan 0.00123
ketertiban 0.00000
listrik 0.01765
bahaya 0.00166
lainnya 0.00000
pungli 0.00000
ilegal 0.00841
lalulintas 0.00014
bencana 0.02040
air 0.47004
pendidikan 0.00000
kebersihan 0.00165
sosial 0.00000
wisata 0.00001
sara 0.00069
pencurian 0.00000
korupsi 0.00000
bbm 0.00000
keuangan 0.00000
